# Setup and imports

In [ ]:
# imports
import numpy as np
from tueplots import bundles, figsizes
import wandb
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd


import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [ ]:
from analysis import sweep2df, plot_typography, stats2string, RED, BLUE, rule_stats2string_per_model, grouped_rule_stats
from llm_non_identifiability.data import A_token, B_token, OPENING_BRACKET_token, OPENING_PARENTHESIS_token, CLOSING_BRACKET_token, CLOSING_PARENTHESIS_token

In [ ]:
USETEX = True

In [ ]:
plt.rcParams.update(bundles.icml2022(usetex=USETEX))
# plt.rcParams.update({
#     'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
#                             r'\usepackage{amsmath}'] # boldsymbol
# })

In [ ]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)


In [ ]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "rule_extrapolation"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

In [ ]:
def get_sweep_stats(sweep_id, file_prefix, save=False, load=True, entity=ENTITY, project=PROJECT):
    api = wandb.Api(timeout=200)
    sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
    filename = f"{file_prefix}_{sweep_id}"
    df,train_loss,val_loss,val_kl,val_accuracy,finised,ood_finised,sos_finised,r1,r2,grammatical,ood_r1,ood_r1_completion,ood_r2,ood_grammatical,sos_r1,sos_r2,sos_grammatical= sweep2df(sweep.runs, filename, save=save, load=load)
    return df, grouped_rule_stats(df)

## baN

In [ ]:
# SWEEP_ID = "y2193u7e"
ban_no_transformer_df, ban_no_transformer_stats = get_sweep_stats(sweep_id="frrbkgg0", file_prefix="ban_no_transformer", save=True, load=False)
ban_transformer_df, ban_transformer_stats = get_sweep_stats(sweep_id="9b9a0xtn", file_prefix="ban_transformer", save=True, load=False)
ban_lstm_df, ban_lstm_stats = get_sweep_stats(sweep_id="vzcozopj", file_prefix="ban_lstm", save=True, load=False)

In [ ]:
ban_df = pd.concat([ban_no_transformer_df, ban_transformer_df])
ban_stats = grouped_rule_stats(ban_df)

## bbaN

In [ ]:
bban_transformer_mamba_df, bban_transformer_mamba_stats = get_sweep_stats(sweep_id="jjjtbh24", file_prefix="bban_transformer_mamba", save=True, load=False)
bban_all_df, bban_all_stats = get_sweep_stats(sweep_id="unc8bv65", file_prefix="bban_all", save=True, load=False)

In [ ]:
bban_df = pd.concat([bban_transformer_mamba_df, bban_all_df])
bban_stats = grouped_rule_stats(bban_df)

## aNbN

In [ ]:
anbn_transformer_df, anbn_transformer_stats = get_sweep_stats(sweep_id="vn4yrcl8", file_prefix="anbn_transformer", save=True, load=False)
anbn_lstm_df, anbn_lstm_stats = get_sweep_stats(sweep_id="t4yzbech", file_prefix="anbn_lstm", save=True, load=False)
anbn_mamba_df, anbn_mamba_stats = get_sweep_stats(sweep_id="o27zaphz", file_prefix="anbn_mamba", save=True, load=False)
anbn_no_transformer_df, anbn_no_transformer_stats = get_sweep_stats(sweep_id="nfrfpkqm", file_prefix="anbn_no_transformer", save=True, load=False)

In [ ]:
anbn_df = pd.concat([anbn_transformer_df, anbn_lstm_df, anbn_mamba_df, anbn_no_transformer_df])
anbn_stats = grouped_rule_stats(anbn_df)

## aNbNcN

In [ ]:
anbncn_df, anbncn_stats = get_sweep_stats(sweep_id="6m1qb70e", file_prefix="anbncn", save=True, load=False)
anbncn_lin_mamba_df, anbncn_lin_mamba_stats = get_sweep_stats(sweep_id="vtji6cx4", file_prefix="anbncn_lin_mamba", save=True, load=False)

In [ ]:
anbncn_df_merged = pd.concat([anbncn_df, anbncn_lin_mamba_df])
anbncn_stats = grouped_rule_stats(anbncn_df_merged)

## Matched brackets and parentheses

In [ ]:
dyck_df, dyck_stats = get_sweep_stats(sweep_id="eruf1l2q", file_prefix="dyck", save=True, load=False)


## Human study

In [ ]:
human_df = pd.read_excel("human_study.xlsx")

# fill nan with empty string
human_df = human_df.fillna("")

# fill "already completed" with empty string
human_df = human_df.replace("already completed", "")

In [ ]:
prompts = list(human_df.columns[1:])
# remove whitespace from beetween characters in the prompts
prompts = ["".join(prompt.split()) for prompt in prompts]

In [ ]:
# overwrite columns names in df
human_df.columns = ["timestamp"] +prompts

In [ ]:
A = A_token.item()
B = B_token.item()
OB = OPENING_BRACKET_token.item()
OP = OPENING_PARENTHESIS_token.item()
CB = CLOSING_BRACKET_token.item()
CP = CLOSING_PARENTHESIS_token.item()

def char2token(char):
    char = char.upper()
    if char == "A":
        return A
    elif char == "B":
        return B
    elif char == "(":
        return OP
    elif char == ")":
        return CP
    elif char == "[":
        return OB
    elif char == "]":
        return CB
    else:
        return None

In [ ]:
# tokenize prompts
prompts_tokenized = [[char2token(c) for c in prompt] for prompt in prompts]



In [ ]:
tokenized_human_df = human_df.copy()

In [ ]:
for prompt, prompt_tokenized in zip(prompts, prompts_tokenized):

    # get the column by prompt
    col = human_df[prompt]
    col_stripped = ["".join(prompt.split()) for prompt in col]
    col_tokenized = [[char2token(c) for c in prompt] for prompt in col_stripped]

    tokenized_human_df[prompt] = col_tokenized



In [ ]:
anbn_r1_human = []
anbn_r2_completion_human = []

dyck_r1_human = []
dyck_r2_completion_human = []

ban_r1_human = []
ban_r2_completion_human = []
from llm_non_identifiability.data import check_same_number_as_bs, check_as_before_bs, check_even_number_of_as, check_begins_with_b, check_matched_brackets, check_matched_parentheses
import torch

for idx, (prompt, prompt_tokenized) in enumerate(zip(prompts, prompts_tokenized)):
    # get the column by prompt
    col = tokenized_human_df[prompt]

    # iterate over the rows
    for completion in col:

        if None in completion:
            continue

        # add the tokenized prompt
        completed_prompt = torch.tensor(prompt_tokenized.copy() + completion)
        completion = torch.tensor(completion)
        # print(completion)

        if idx <= 4: # anbn
            anbn_r1_human.append(check_same_number_as_bs(completed_prompt))
            anbn_r2_completion_human.append(True if len(completion)==0 else check_as_before_bs(completion))
        elif idx <=9    : # dyck
            dyck_r1_human.append(check_matched_brackets(completed_prompt[2:]))
            dyck_r2_completion_human.append(True if len(completion)==0 else check_matched_parentheses(completion))
        else: # ban
            ban_r1_human.append(check_even_number_of_as(completed_prompt))
            ban_r2_completion_human.append(True if len(completion)==0 else check_begins_with_b(completion))



In [ ]:
# print accuracies
print(f"anbn R1: {np.mean(anbn_r1_human)}")
print(f"anbn R2: {np.mean(anbn_r2_completion_human)}")
print(f"dyck R1: {np.mean(dyck_r1_human)}"
)
print(f"dyck R2: {np.mean(dyck_r2_completion_human)}")
print(f"ban R1: {np.mean(ban_r1_human)}")
print(f"ban R2: {np.mean(ban_r2_completion_human)}")

In [ ]:
human_stats = {}
human_stats["baN"] = {
    "ood_rule_1": np.mean(ban_r1_human),
    "ood_rule_2_completion": np.mean(ban_r2_completion_human)
}

human_stats["aNbN"] = {
    "ood_rule_1": np.mean(anbn_r1_human),
    "ood_rule_2_completion": np.mean(anbn_r2_completion_human)
}

human_stats["Dyck"] = {
    "ood_rule_1": np.mean(dyck_r1_human),
    "ood_rule_2_completion": np.mean(dyck_r2_completion_human)
}

## Chance levels

In [ ]:
chance_stats = {}
chance_stats["baN"] = {
    "ood_rule_1": 0.5,
    "ood_rule_2_completion": 0.5
}
chance_stats["aNbN"] = {
    "ood_rule_1": 0.153,
    "ood_rule_2_completion": 0.75
}
# chance_stats["Dyck"] = {
#     "ood_rule_1": 0.5,
#     "ood_rule_2_completion": 0.5
# }
chance_stats["bbaN"] = {
    "ood_rule_1": 0.5,
    "ood_rule_2_completion": 0.5
}

chance_stats["aNbNcN"] = {
    "ood_rule_1": 0.00945,
    "ood_rule_2_completion": 0.5925
}




# Plots

## helper functions


In [ ]:
def plot_loss_vs_rules(df, stats, cmap="coolwarm", TICK_PADDING=2, LABELPAD=1, filename=None):

    colors = {
        "transformer": "tab:blue",
        "lstm": "tab:orange",
        "linear": "tab:green",
        "mamba": "tab:red"
        }


    fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])
    ax = fig.add_subplot(121)
    ax.grid(True, which="both", ls="-.")
    ax.set_axisbelow(True)
    for model in stats["val_loss"].groups.keys():

        im = ax.scatter(df[df.model == model].min_val_loss,
                        100 * df[df.model == model].ood_rule_1_accuracy4min_val_loss, c=colors[model], label=model)
    ax.set_ylabel("R1 \%", labelpad=LABELPAD)
    ax.set_xlabel("Minimum test loss", labelpad=LABELPAD)
    # plt.legend()
    ax.tick_params(axis='both', which='major', pad=TICK_PADDING)
    ax = fig.add_subplot(122)
    ax.grid(True, which="both", ls="-.")
    ax.set_axisbelow(True)
    for model in stats["val_loss"].groups.keys():
        im = ax.scatter(df[df.model == model].min_val_loss,
                        100 * df[df.model == model].ood_rule_2_completion_accuracy4min_val_loss, c=colors[model], label=model)
    ax.set_ylabel("R2 completion (\%)", labelpad=LABELPAD)
    ax.set_xlabel("Minimum test loss", labelpad=LABELPAD)
    plt.legend(loc='lower center')
    ax.tick_params(axis='both', which='major', pad=TICK_PADDING)

    if filename is not None:
        plt.savefig(f"{filename}.svg")

## baN

In [ ]:
plot_loss_vs_rules(ban_df, ban_stats, filename="ban_loss_vs_rules")

In [ ]:
rule_stats2string_per_model(ban_stats)

## bbaN

In [ ]:
plot_loss_vs_rules(bban_df, bban_stats, filename="bban_loss_vs_rules")

In [ ]:
rule_stats2string_per_model(bban_stats)

## aNbN

In [ ]:
plot_loss_vs_rules(anbn_df, anbn_stats, filename="anbn_loss_vs_rules")

In [ ]:
rule_stats2string_per_model(anbn_stats)

## aNbNcN

In [ ]:
plot_loss_vs_rules(anbncn_df, anbncn_stats, filename="anbncn_loss_vs_rules")

In [ ]:
rule_stats2string_per_model(anbncn_stats)

## Matched brackets and parentheses

In [ ]:
plot_loss_vs_rules(dyck_df, dyck_stats, filename="dyck_loss_vs_rules")

In [ ]:
rule_stats2string_per_model(dyck_stats)

## Plot for all languages

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"


stats_dict = {
    "baN": ban_stats,
    "bbaN": bban_stats,
    "aNbN": anbn_stats,
    "aNbNcN": anbncn_stats,
    "Dyck": dyck_stats,
}

colors = {
        "transformer": "tab:blue",
        "lstm": "tab:orange",
        "linear": "tab:green",
        "mamba": "tab:red",
    "human" : "black",
    # "chance": "purple"
        }


x_pos = list(range(len(stats_dict)))
x_offsets = [-0.45, -0.3, -.15, .15, 0.3, .45]
x_factor = 1/ len(stats_dict)

fig = plt.figure(figsize=figsizes.neurips2022(nrows=1, ncols=2, tight_layout=True, rel_width=2)['figure.figsize'])

ax = fig.add_subplot(121)
# ax.grid(True, which="both", ls="-.")
# ax.set_axisbelow(True)

models =  colors.keys()
for x, (grammar, stats) in enumerate(stats_dict.items()):

    if grammar in chance_stats.keys():
        chance = chance_stats[grammar]["ood_rule_1"]
        ax.axhline(100*chance, xmin=x_factor*x, xmax=x_factor*(x+1),  color='black', linestyle='--', linewidth=0.5)

    for i, model in enumerate(models):

        if model == "human":
            if grammar in human_stats.keys():
                mean = human_stats[grammar]["ood_rule_1"]
                std = 0
            else:
                continue
        # elif model == "chance":
        #     if grammar in chance_stats.keys():
        #         mean = chance_stats[grammar]["ood_rule_1"]
        #         std = 0
        #     else:
        #         continue
        else:
            if model not in stats["ood_rule_1"].groups.keys():
                continue
            mean = stats["ood_rule_1"].get_group(model).mean()
            std = stats["ood_rule_1"].get_group(model).std()


        ax.errorbar(x + x_offsets[i], 100 * mean, yerr=10 * std, fmt='o', label=model, c=colors[model])



    ax.axvline(x+0.5, color='black', linestyle='--', linewidth=0.5)


ax.set_ylabel("R1 (\%)", labelpad=LABELPAD)

# set xtick names
ax.set_xticks(x_pos)
ax.set_xticklabels(stats_dict.keys())

ax2 = fig.add_subplot(122)

for x, (grammar, stats) in enumerate(stats_dict.items()):

    if grammar in chance_stats.keys():
        chance = chance_stats[grammar]["ood_rule_2_completion"]
        ax2.axhline(100*chance, xmin=x_factor*x, xmax=x_factor*(x+1),  color='black', linestyle='--', linewidth=0.5)


    for i, model in enumerate(models):

        if model == "human":
            if grammar in human_stats.keys():
                mean = human_stats[grammar]["ood_rule_2_completion"]
                std = 0
            else:
                continue
        # elif model == "chance":
        #     if grammar in chance_stats.keys():
        #         mean = chance_stats[grammar]["ood_rule_2_completion"]
        #         std = 0
        #     else:
        #         continue
        else:
            if model not in stats["ood_rule_2_completion"].groups.keys():
                continue
            mean = stats["ood_rule_2_completion"].get_group(model).mean()
            std = stats["ood_rule_2_completion"].get_group(model).std()

        ax2.errorbar(x+x_offsets[i], 100*mean, yerr=10*std, fmt='o',c=colors[model], label = model)


    ax2.axvline(x+0.5, color='black', linestyle='--', linewidth=0.5)


# set xtick names
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stats_dict.keys())
ax2.set_ylabel("R2 completion (\%)", labelpad=LABELPAD)


# plot legends, only one form each model with a dot as the symbol


handles = [mlines.Line2D([], [], color=colors[model], marker='o', label=model.capitalize()) for model in models]
ax2.legend(handles=handles, loc="center right", bbox_to_anchor=(1.75, 0.5))#, loc='upper center',  ncol=4)



# plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)

plt.savefig("ood_summary.svg")
